## OSTEWG 2019 Challenge - ryanofarizona

## WAMI Data Association and Tracking

### General imports

In [1]:
import numpy as np
import os

### Get path to the data

In [2]:
data_dir = os.path.abspath(os.getenv('OSTEWG_DATA_DIR', '../../Stone-Soup-Data/wami-2019-ostewg-challenge'))
alltruth_filename = os.path.join(data_dir, 'alltruth.txt')
truth_filename = os.path.join(data_dir, 'truth.txt')
detections_filename = os.path.join(data_dir, 'detections.txt')

### Configure truth readers

In [3]:
import csv

from stonesoup.reader.generic import CSVGroundTruthReader

class UoLWAMIDataDialect(csv.Dialect):
    delimiter=' '
    skipinitialspace=True
    quoting=csv.QUOTE_NONE
    lineterminator='\r\n'

alltruth_reader = CSVGroundTruthReader(
    alltruth_filename,
    state_vector_fields=["x", "y"],
    time_field="time",
    timestamp=True,
    path_id_field="id",
    csv_options={'dialect': UoLWAMIDataDialect,
                 'fieldnames': ['time', 'x', 'y', 'id']})

truth_reader = CSVGroundTruthReader(
    truth_filename,
    state_vector_fields=["x", "y"],
    time_field="time",
    timestamp=True,
    path_id_field="id",
    csv_options={'dialect': UoLWAMIDataDialect,
                 'fieldnames': ['time', 'x', 'y', 'id']})

### Load truth tracks

In [4]:
alltruth_paths = set()
alltruth_times = set()
for time_dt64, path_states in alltruth_reader.groundtruth_paths_gen():
    alltruth_times.add(time_dt64)
    alltruth_paths.update(path_states)

truth_paths = set()
truth_times = set()
for time_dt64, path_states in truth_reader.groundtruth_paths_gen():
    truth_times.add(time_dt64)
    truth_paths.update(path_states)

### Inspect the structure of the truth data

In [5]:
# get the track count
len(alltruth_paths)

144

In [6]:
# get start time, end time, duration
start_dt64 = min(alltruth_times)
end_dt64 = max(alltruth_times)
duration_s = (end_dt64 - start_dt64).total_seconds()
print(start_dt64, end_dt64, duration_s, sep=', ')

1970-01-01 00:00:00, 1970-01-01 00:05:00, 300.0


In [7]:
# look at a single track
singletruth = next(iter(alltruth_paths))

In [8]:
type(singletruth)

stonesoup.types.groundtruth.GroundTruthPath

In [9]:
# get the track id
singletruth.id

'4.3000000e+01'

In [10]:
# get the length of the states list
len(singletruth.states)

1256

In [11]:
# get the times of all the states
singletruth_times = [ss.timestamp for ss in singletruth.states]
# look at the first and last times
print(singletruth_times[0], singletruth_times[-1])

1970-01-01 00:00:49 1970-01-01 00:05:00


In [12]:
# time difference stats
singletruth_time_diffs = np.array([dd.total_seconds() for dd in np.diff(singletruth_times)])
diff_seconds, diff_counts = np.unique(singletruth_time_diffs, return_counts=True)
{diff_time:diff_count for diff_time, diff_count in zip(diff_seconds, diff_counts)}

{0.2: 1255}

### Look into the truth tracks that were not detected
`alltruth_detections` includes all truth, including those objects that were not detected. `truth_detections` only includes truth for those objects that were detected and at the times that they were detected. It is worth looking into the differences to shed light on why they might not have been detected.

In [13]:
alltruth_id_set = {tt.id for tt in alltruth_paths}
truth_id_set = {tt.id for tt in truth_paths}
undetected_id_set = alltruth_id_set.difference(truth_id_set)
undetected_id_set

{'1.0400000e+02',
 '1.0700000e+02',
 '1.1200000e+02',
 '1.1500000e+02',
 '1.1900000e+02',
 '1.2000000e+02',
 '1.2800000e+02',
 '1.3100000e+02',
 '1.3200000e+02',
 '1.3900000e+02',
 '1.4000000e+01',
 '1.4000000e+02',
 '1.4200000e+02',
 '1.4700000e+02',
 '1.4800000e+02',
 '1.5000000e+02',
 '2.1000000e+01',
 '2.2000000e+01',
 '2.4000000e+01',
 '3.1000000e+01',
 '3.2000000e+01',
 '3.4000000e+01',
 '3.7000000e+01',
 '4.3000000e+01',
 '4.5000000e+01',
 '4.7000000e+01',
 '4.9000000e+01',
 '5.0000000e+01',
 '5.2000000e+01',
 '5.3000000e+01',
 '5.4000000e+01',
 '5.7000000e+01',
 '6.2000000e+01',
 '6.5000000e+01',
 '6.7000000e+01',
 '6.8000000e+01',
 '7.3000000e+01',
 '7.7000000e+01',
 '7.8000000e+01',
 '8.5000000e+01',
 '9.3000000e+01',
 '9.4000000e+01',
 '9.5000000e+01',
 '9.7000000e+01'}

'c2f86049-da98-4359-ac4c-3c10ff901e38'